In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import numpy as np
import pandas as pd

from tqdm import tqdm
import random
import math
import json

import time
import os

In [2]:
NEG_SAMPLE_SIZE = 10
EPOCH = 100
BATCH_SIZE = 32
BATCH_SIZE_TEST = 32
VALIDATION_INTERVAL = 100
SAVE_ITERATION = 100
EMBEDDING_SIZE = 10
GAMMA = 12
LEARNING_RATE = 0.0001
REGULARIZATION = 0
ADVERSARIAL_TEMPERATURE  = 1 #In self-adversarial sampling, we do not apply back-propagation on the sampling weight
NUM_WORKERS = 6

double_entity_embedding = True # True if RotatE, ComplEx
double_relation_embedding = True # True if ComplEx
uni_weight = True

NAME_PATH = 'log//'+ time.ctime()
os.mkdir(NAME_PATH)


In [3]:
variables = {'NEG_SAMPLE_SIZE': NEG_SAMPLE_SIZE,
             'EPOCH': EPOCH,
             'BATCH_SIZE': BATCH_SIZE,
             'BATCH_SIZE_TEST': BATCH_SIZE_TEST,
             'VALIDATION_INTERVAL': VALIDATION_INTERVAL,
             'SAVE_ITERATION': SAVE_ITERATION,
             'EMBEDDING_SIZE': EMBEDDING_SIZE,
             'GAMMA':  GAMMA,
             'LEARNING_RATE': LEARNING_RATE,
             'REGULARIZATION': REGULARIZATION,
             'ADVERSARIAL_TEMPERATURE': ADVERSARIAL_TEMPERATURE,
             'NUM_WORKERS': NUM_WORKERS}

with open(NAME_PATH+'//varibles.txt', 'w') as var_file:
            json.dump(variables, var_file)

#### Предобработка данных

In [ ]:
df = pd.read_csv ('clear_data_realbank.csv')
df.drop('TRANS_DETAIL', inplace=True, axis=1)
df.rename(columns = {'RETAILER' : 'tail', 'CustomerKey' : 'head',
                        'MCC' : 'tail_type', 'AMOUNT_EQ' : 'relation'   }, 
                            inplace = True) 
# некоторые id магазинов имели разные категории. Здесь id c разл категорией приравниваются к -2 и затем удаляются
for i in (range(min(df.tail_type.unique()),max(df.tail_type.unique())+1)):
    for j in range(i+1,max(df.tail_type.unique())+1):
        for k in (set(df[df.tail_type==i]['tail'].unique())&set(df[df.tail_type==j]['tail'].unique())):
            df.loc[df['tail'] == k,'tail_type'] = -2
df = df[df.tail_type != -2].reset_index(drop=True)

# траты разбиваются на 8 категорий и в дальнейшем будут характеризировать отношения
df.relation = pd.qcut(df.relation, q=8, 
        labels=["small", "medium_small", "medium_small_2", 'medium_1', 'medium_2', 'medium_large_2', 'medium_large', 'large'])

# удаление непопулярных категорий магазинов
top_mcc = list(df.tail_type.value_counts()[:10].rename_axis('unique_values').reset_index(name='counts')['unique_values'])
df = df[df.tail_type.isin(top_mcc)].reset_index(drop=True)
df = df.sort_values('tail_type')

# присвоение уникальным id пользователей и магазинов чисел от 0 до len(уникальных id)
df['head'] = pd.factorize(df['head'])[0]
max_person = max(df['head'])
df['tail'] = pd.factorize(df['tail'])[0] + max_person + 1

# словарь с типами сущностей и списком их id
dict_id = {}
dict_id['person'] = list(range(0,max(df['head'])+1))
for tail_type in df['tail_type'].unique():
    dict_id[tail_type] = list(range(min(df[df.tail_type==tail_type]['tail']),(max(df[df.tail_type==tail_type]['tail'])+1)))

# присвоение чисел отношениям
dict_rel = {key: idx for idx,key in enumerate(pd.factorize(df.relation)[1].categories)}
df['relation'] = df['relation'].apply(lambda x: dict_rel[x])

# создание негативных сущностей 
# neg_head --> tail      neg_tail --> head
df['neg_head'] = [random.sample(dict_id['person'],NEG_SAMPLE_SIZE) for _ in range(len(df))]
df['neg_tail'] = [random.sample(dict_id[i], NEG_SAMPLE_SIZE) for i in df.tail_type]

# разбиение на выборки по времени
train = df[(df.tstmp >= '2018-01-01 00:00:00+03:00') & (df.tstmp < '2018-08-01 00:00:00+03:00')]
train.drop('tstmp', inplace=True, axis=1, errors='ignore')
valid = df[(df.tstmp >= '2018-08-01 00:00:00+03:00') & (df.tstmp < '2018-10-01 00:00:00+03:00')]
valid.drop('tstmp', inplace=True, axis=1, errors='ignore')
test = df[(df.tstmp >= '2018-10-01 00:00:00+03:00') & (df.tstmp < '2019-01-01 00:00:00+03:00')]
test.drop('tstmp', inplace=True, axis=1, errors='ignore')

In [5]:
print(f'TRAIN: {round(len(train)/len(df)*100)}%')
print(f'VALID: {round(len(valid)/len(df)*100)}%')
print(f'TEST: {round(len(test)/len(df)*100)}%')

TRAIN: 58%
VALID: 17%
TEST: 25%


In [6]:
train_count, train_true_head, train_true_tail = defaultdict(lambda: 4), defaultdict(list), defaultdict(list)
for i in tqdm(train.index):
    head, relation, tail = train.loc[i,'head'], train.loc[i,'relation'],  train.loc[i,'tail']
    train_count[(head, relation)] += 1
    train_count[(tail, -relation-1)] += 1
    train_true_head[(relation, tail)].append(head)
    train_true_tail[(head, relation)].append(tail)

train['subsampling_weight'] = [(1/(train_count[(train.loc[i,'head'], train.loc[i,'relation'])] 
                                + train_count[(train.loc[i,'tail'], -train.loc[i,'relation']-1)]))**(1/2)
                                                                                for i in train.index]

nentity = len(df['tail'].unique())+len(df['head'].unique())
nrelation = len(df['relation'].unique())

100%|██████████| 1795880/1795880 [00:58<00:00, 30620.96it/s]


#### Модель

In [8]:
class KGEModel(nn.Module):
    def __init__(self, nentity, nrelation, embedding_size, gamma, evaluator,
                 double_entity_embedding=False, double_relation_embedding=False, epsilon = 2.0):
        super(KGEModel, self).__init__()
        
        self.gamma = nn.Parameter(
            torch.Tensor([gamma]), 
            requires_grad=False
        )
        
        self.embedding_range = nn.Parameter(
            torch.Tensor([(self.gamma.item() + epsilon) / embedding_size]), 
            requires_grad=False
        )
        
        self.entity_dim = embedding_size*2 if double_entity_embedding else embedding_size
        self.relation_dim = embedding_size*2 if double_relation_embedding else embedding_size
        
        self.entity_embedding = nn.Parameter(torch.zeros(nentity, self.entity_dim))
        nn.init.uniform_(
            tensor=self.entity_embedding, 
            a=-self.embedding_range.item(), 
            b=self.embedding_range.item()
        )
        
        self.relation_embedding = nn.Parameter(torch.zeros(nrelation, self.relation_dim))
        nn.init.uniform_(
            tensor=self.relation_embedding, 
            a=-self.embedding_range.item(), 
            b=self.embedding_range.item()
        )

        self.evaluator = evaluator
        
    def forward(self, head, tail, relation, neg_head, neg_tail):
        head_E = torch.index_select(
                self.entity_embedding, 
                dim=0, 
                index=head
            ).unsqueeze(1)
        
        relation_E = torch.index_select(
                self.relation_embedding, 
                dim=0, 
                index=relation
            ).unsqueeze(1)

        tail_E = torch.index_select(
                self.entity_embedding, 
                dim=0, 
                index=tail
            ).unsqueeze(1)

        neg_head_E = torch.index_select(
                self.entity_embedding, 
                dim=0, 
                index=neg_head.view(-1)
            ).view(neg_head.size(0), neg_head.size(1), -1)  #batch_size, negative_sample_size
            
        neg_tail_E  = torch.index_select(
                self.entity_embedding, 
                dim=0, 
                index=neg_tail.view(-1)
            ).view(neg_tail.size(0), neg_tail.size(1), -1) 
            
        #TransE
        positive_score = (head_E + relation_E) - tail_E
        negative_tail_score = (head_E + relation_E) - neg_tail_E
        negative_head_score = neg_head_E + (relation_E - tail_E)

        positive_score = self.gamma.item() - torch.norm(positive_score, p=1, dim=2)
        negative_tail_score = self.gamma.item() - torch.norm(negative_tail_score, p=1, dim=2)
        negative_head_score = self.gamma.item() - torch.norm(negative_head_score, p=1, dim=2)
        
        return positive_score, negative_tail_score, negative_head_score

#### Метрика

In [9]:
class Evaluator:
    def eval(self, input_dict):
        y_pred_pos, y_pred_neg = input_dict['y_pred_pos'], input_dict['y_pred_neg']
        y_pred = torch.cat([y_pred_pos.view(-1,1), y_pred_neg], dim = 1)
        argsort = torch.argsort(y_pred, dim = 1, descending = True)
        ranking_list = torch.nonzero(argsort == 0, as_tuple=False)
        ranking_list = ranking_list[:, 1] + 1
        hits1_list = (ranking_list <= 1).to(torch.float)
        hits3_list = (ranking_list <= 3).to(torch.float)
        hits10_list = (ranking_list <= 10).to(torch.float)
        mrr_list = 1./ranking_list.to(torch.float)

        return mrr_list, hits1_list, hits3_list, hits10_list

#### TRAIN

In [10]:
class MyDataset(Dataset):

  def __init__(self,table,mode='train'):

    self.mode = mode
    self.head = torch.tensor(np.array(table['head']))
    self.tail = torch.tensor(np.array(table['tail']))
    self.relation = torch.tensor(np.array(table['relation']))
    self.neg_head = torch.tensor(np.array(list(table['neg_head'])))
    self.neg_tail = torch.tensor(np.array(list(table['neg_tail'])))

    if mode=='train':
      self.subsampling_weight = torch.tensor(np.array(table['subsampling_weight']))

  def __len__(self):
    return len(self.head)
  
  def __getitem__(self,idx):
    if self.mode == 'train':
      return self.head[idx],self.tail[idx],self.relation[idx], self.neg_head[idx],self.neg_tail[idx], self.subsampling_weight[idx]
    else:
      return self.head[idx],self.tail[idx],self.relation[idx], self.neg_head[idx],self.neg_tail[idx]

In [ ]:
evaluator = Evaluator()
kge_model = KGEModel(
        nentity=nentity,
        nrelation=nrelation,
        embedding_size=EMBEDDING_SIZE,
        gamma=GAMMA,
        double_entity_embedding=double_entity_embedding,
        double_relation_embedding=double_relation_embedding,
        evaluator=evaluator
    )
kge_model = kge_model.cuda()
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, kge_model.parameters()), 
            lr=LEARNING_RATE
        )

log, mrr, hits1, hits3, hits10 = [], [], [], [], []

for num_epoch in (range(EPOCH)):
    train_loader=DataLoader(MyDataset(train),batch_size=BATCH_SIZE,shuffle=False,num_workers=NUM_WORKERS)
    iteration = math.ceil(len(train)/BATCH_SIZE) 
    for i in tqdm(range(iteration)):
        kge_model.train()
        optimizer.zero_grad()

        head,tail,relation,neg_head,neg_tail,subsampling_weight = next(iter(train_loader))

        head = head.cuda()
        tail = tail.cuda()
        relation = relation.cuda()
        neg_head = neg_head.cuda()
        neg_tail = neg_tail.cuda()
        subsampling_weight = subsampling_weight.cuda()

        positive_score, negative_tail_score, negative_head_score = kge_model(head, tail, relation, neg_head, neg_tail)

        positive_score = F.logsigmoid(positive_score).squeeze(dim = 1)
        if ADVERSARIAL_TEMPERATURE!=0.0:
            #In self-adversarial sampling, we do not apply back-propagation on the sampling weight
            negative_tail_score = (F.softmax(negative_tail_score * ADVERSARIAL_TEMPERATURE, dim = 1).detach() 
                              * F.logsigmoid(-negative_tail_score)).sum(dim = 1)
            negative_head_score = (F.softmax(negative_head_score * ADVERSARIAL_TEMPERATURE, dim = 1).detach() 
                              * F.logsigmoid(-negative_head_score)).sum(dim = 1)
        else:
            negative_tail_score = F.logsigmoid(-negative_tail_score).mean(dim = 1)
            negative_head_score = F.logsigmoid(-negative_head_score).mean(dim = 1)

        if uni_weight:
            positive_sample_loss = - positive_score.mean()
            negative_sample_tail_loss = - negative_tail_score.mean()
            negative_sample_head_loss = - negative_head_score.mean()
        else:
            positive_sample_loss = - (subsampling_weight * positive_score).sum()/subsampling_weight.sum()
            negative_sample_tail_loss = - (subsampling_weight * negative_tail_score).sum()/subsampling_weight.sum()
            negative_sample_head_loss = - (subsampling_weight * negative_head_score).sum()/subsampling_weight.sum()

        loss = (2*positive_sample_loss + negative_sample_tail_loss+negative_sample_head_loss)/4

        if REGULARIZATION != 0.0:
            #Use L3 regularization for ComplEx and DistMult
            REGULARIZATION = REGULARIZATION * (
                kge_model.entity_embedding.norm(p = 3)**3 + 
                kge_model.relation_embedding.norm(p = 3).norm(p = 3)**3
            )
            loss = loss + REGULARIZATION
        
        loss.backward()

        log.append(float(loss))

        optimizer.step()
    
# VALID
        if (iteration+1)%VALIDATION_INTERVAL == 0:
            valid_loader=DataLoader(MyDataset(valid,mode='test'),batch_size=BATCH_SIZE,shuffle=False,num_workers=NUM_WORKERS)
            iteration = math.ceil(len(valid)/BATCH_SIZE_TEST) 
            for i in range(iteration):
                kge_model.eval()

                head,tail,relation,neg_head,neg_tail = next(iter(valid_loader))

                head = head.cuda()
                tail = tail.cuda()
                relation = relation.cuda()
                neg_head = neg_head.cuda()
                neg_tail = neg_tail.cuda()

                with torch.no_grad():
                    positive_score, negative_tail_score, negative_head_score = kge_model(head, tail, relation, neg_head, neg_tail)

                    mrr_tail, hits1_tail, hits3_tail, hits10_tail = kge_model.evaluator.eval({'y_pred_pos': positive_score, 'y_pred_neg': negative_tail_score})
                    mrr_head, hits1_head, hits3_head, hits10_head = kge_model.evaluator.eval({'y_pred_pos': positive_score, 'y_pred_neg': negative_head_score})
                    mrr_tail, hits1_tail, hits3_tail, hits10_tail = mrr_tail.mean(), hits1_tail.mean(), hits3_tail.mean(), hits10_tail.mean()
                    mrr_head, hits1_head, hits3_head, hits10_head = mrr_head.mean(), hits1_head.mean(), hits3_head.mean(), hits10_head.mean()
                    mrr.append(float((mrr_tail+mrr_head)/2))
                    hits1.append(float((hits1_tail+hits1_head)/2))
                    hits3.append(float((hits3_tail+hits3_head)/2))
                    hits10.append(float((hits10_tail+hits10_head)/2))

        if (num_epoch+1)%SAVE_ITERATION == 0:
            with open(NAME_PATH+'//log.txt', 'w') as log_file, open(NAME_PATH+'//mrr.txt', 'w') as mrr_file:
                json.dump(log, log_file)
                json.dump(mrr, mrr_file)
            plt.clf()
            figure, axis = plt.subplots(2, 2)
            axis[0,0].plot(log)
            axis[0,0].set_title("TRAIN LOSS")
  
            axis[0,1].plot(mrr)
            axis[0,1].set_title("VALID MRR")

            axis[1,0].plot(hits1)
            axis[1,0].set_title("VALID HITS@1")

            axis[1,1].plot(hits3)
            axis[1,1].set_title("VALID HITS@1")

            figure.tight_layout()
        
            plt.savefig(NAME_PATH+'//result.png', dpi=300, facecolor='w', edgecolor='w')

#### TEST

In [60]:
mrr_test, hits1_test, hits3_test, hits10_test = [], [], [], []
test_loader=DataLoader(MyDataset(test,mode='test'),batch_size=BATCH_SIZE,shuffle=False,num_workers=NUM_WORKERS)
iteration = math.ceil(len(test)/BATCH_SIZE_TEST) 
for i in range(iteration):
        kge_model.eval()
        
        head,tail,relation,neg_head,neg_tail = next(iter(test_loader))

        head = head.cuda()
        tail = tail.cuda()
        relation = relation.cuda()
        neg_head = neg_head.cuda()
        neg_tail = neg_tail.cuda()

        with torch.no_grad():
            positive_score, negative_tail_score, negative_head_score = kge_model(head, tail, relation, neg_head, neg_tail)

            mrr_tail, hits1_tail, hits3_tail, hits10_tail = kge_model.evaluator.eval({'y_pred_pos': positive_score, 'y_pred_neg': negative_tail_score})
            mrr_head, hits1_head, hits3_head, hits10_head = kge_model.evaluator.eval({'y_pred_pos': positive_score, 'y_pred_neg': negative_head_score})
            mrr_tail, hits1_tail, hits3_tail, hits10_tail = mrr_tail.mean(), hits1_tail.mean(), hits3_tail.mean(), hits10_tail.mean()
            mrr_head, hits1_head, hits3_head, hits10_head = mrr_head.mean(), hits1_head.mean(), hits3_head.mean(), hits10_head.mean()
            mrr_test.append(float((mrr_tail+mrr_head)/2))
            hits1_test.append(float((hits1_tail+hits1_head)/2))
            hits3_test.append(float((hits3_tail+hits3_head)/2))
            hits10_test.append(float((hits10_tail+hits10_head)/2))

dict_metrics = {'MRR': np.array(mrr_test).mean(),
                'HITS@1': np.array(hits1_test).mean(),
                'HITS@3': np.array(hits3_test).mean(),
                'HITS@10': np.array(hits10_test).mean()}

with open(NAME_PATH+'//metrics.txt', 'w') as var_file:
            json.dump(dict_metrics, var_file)

In [ ]:
# # Чтение
# with open('log.txt', 'r') as log_file, open('mrr.txt', 'r') as mrr_file:
#     log = json.load(log_file)
#     mrr = json.load(mrr_file)